<a href="https://colab.research.google.com/github/Yewon9/STT/blob/main/Wav2Vec2_Fine_Tuning_Disordered_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets soundfile

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

# DATA

In [4]:
df = pd.read_csv('disorder_eda.csv')
df

,Path,Text,Audio_Length,Cleaned_Text,Text_Length
0,/content/drive/MyDrive/project/data/Training/w...,미영이랑 나연이는 단짝입니다. 미영이와 나연이는 노래하며 놉니다. 마루 위에 나란히...,772.163,미영이랑 나연이는 단짝입니다. 미영이와 나연이는 노래하며 놉니다. 마루 위에 나란히...,2038
1,/content/drive/MyDrive/project/data/Training/w...,거울 안경 전화 신발 나무 가위 사과 모자 풍선 당근 연필 기차 날개 창문 동생 약...,206.577,거울 안경 전화 신발 나무 가위 사과 모자 풍선 당근 연필 기차 날개 창문 동생 약...,215
2,/content/drive/MyDrive/project/data/Training/w...,나는 바지를 입고 단추를 채웁니다. 책상 위에 가방이 있습니다. 가방에 사탕과 연필...,709.480,나는 바지를 입고 단추를 채웁니다. 책상 위에 가방이 있습니다. 가방에 사탕과 연필...,335
3,/content/drive/MyDrive/project/data/Training/w...,새 해 배 개 집 일 입 이 꿀 불 귤 물 피 쥐 비 귀 콩 돈 공 손 풀 붓 뿔 ...,1490.877,새 해 배 개 집 일 입 이 꿀 불 귤 물 피 쥐 비 귀 콩 돈 공 손 풀 붓 뿔 ...,299
4,/content/drive/MyDrive/project/data/Training/w...,바지 단추 책상 가방 사탕 연필 자동차 동물원 엄마 뽀뽀 호랑이 꼬리 코끼리 땅콩 ...,276.963,바지 단추 책상 가방 사탕 연필 자동차 동물원 엄마 뽀뽀 호랑이 꼬리 코끼리 땅콩 ...,89
...,...,...,...,...,...
309,/content/drive/MyDrive/project/data/Training/w...,제 나이는 서른 일곱 살입니다. 저는 용 띠입니다. 제가 태어난 곳은 동해시이고 주...,920.580,제 나이는 서른 일곱 살입니다. 저는 용 띠입니다. 제가 태어난 곳은 동해시이고 주...,724
310,/content/drive/MyDrive/project/data/Training/w...,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,3816.600,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,4211
311,/content/drive/MyDrive/project/data/Training/w...,거짓말을 하는 것이 분명합니다. 고기는 좋아하지만 생선은 싫어한다. 기차를 탈 분은...,2753.132,거짓말을 하는 것이 분명합니다. 고기는 좋아하지만 생선은 싫어한다. 기차를 탈 분은...,200
312,/content/drive/MyDrive/project/data/Training/w...,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,4320.615,100만원 태국 돈으로 얼마야? 10월 14일 로또 당첨자 알려 줘 10월 16일 ...,4190


In [5]:
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['Path', 'Text', 'Audio_Length', 'Cleaned_Text', 'Text_Length'],
    num_rows: 314
})
